In [1]:
import requests # Apache License 2.0
from requests.auth import HTTPBasicAuth

import uuid     # in python
import base64   # in python
import copy     # in python
import yaml     # MIT


from app.utils import print_shelldescriptors, print_submodels, get_submodel_template, get_submodel_element_blob_template

In [2]:
# --- variables ---
with open('provider_cfg.yaml', 'r') as file:
    consumer_cfg = yaml.safe_load(file)

# - repository -
credentials_repository = consumer_cfg['submodel-repository']['credentials']
endpoint_repository    = consumer_cfg['submodel-repository']['endpoint']

# - registry -
header_registry   = consumer_cfg['registry']['header']
endpoint_registry = consumer_cfg['registry']['endpoint']
# to add the endpoint where we can negotiate for this asset
url_edc_provider_control_plane_base = consumer_cfg['provider-edc-control-plane']['endpoint']

# - "identities" -
edc_consumer_bpn = consumer_cfg['trusted-consumers-bpn']['machine_builder_A']

# --- helper functions ---
# we require the AAS id to be b64 encoded (for example because some ids contain urls)
def to64str(str_in):
    b64 = base64.b64encode(str_in.encode('utf-8')).decode('utf-8')
    return b64

In [ ]:
# obtain 'all' (w.r.t. the default tenant) registred shell descriptors
res_descriptors = requests.get(endpoint_registry + '/shell-descriptors', headers=header_registry)
print_shelldescriptors(res_descriptors)

### Create a new Asset -> Digital Twin

In [ ]:
# lets create a shell-descriptor according to:
# https://eclipse-tractusx.github.io/docs-kits/next/kits/Digital%20Twin%20Kit/Software%20Development%20View/dt-kit-software-development-view

urn_uuid   = uuid.uuid4() 
asset_name = "demo-test-fr"
print("Asset " + asset_name + " with UUID: " + str(urn_uuid))
manufacturerPartId = "123-456-789-10"      
BPN_consumer       = edc_consumer_bpn # consumer(s) we entrust with this asset data


registrate_asset_body = {
  "id":      "urn:uuid:" + str(urn_uuid),
  "idShort": asset_name,                  # this has to be unique for a reason aswell
  "globalAssetId": str(urn_uuid),         # <- not in the specification
  "specificAssetIds": [
    {
      "name": "manufacturerPartId",
      "value": manufacturerPartId,
      "externalSubjectId": {
        "type": "ExternalReference",
        "keys": [
          {
            "type": "GlobalReference",
            "value": BPN_consumer       # <- necessary
          }
        ],
      },
    }
  ],
  "submodelDescriptors": [],
}

# post into the registry:
res_add_descriptor = requests.post(endpoint_registry + '/shell-descriptors', headers=header_registry, json=registrate_asset_body)
res_add_descriptor

In [44]:
# ----------------------------------------------------------------------------------------------------------------------

### Create Submodel with its Submodel-Elements

In [76]:
# create a the corresponding submodel in the SUBMODEL-repository
submodel_name = "TSsubmodel-demo"
submodel_uuid = uuid.uuid4()

# Submodel-Element
submodel_element_name  = "Blob-1"
submodel_element_uuid  = uuid.uuid4()
submodel_element_value = "demo value fr 10:32"

In [ ]:
# --- fill tempaltes ---
# submodel:
submodel = get_submodel_template()

submodel["idShort"] = submodel_name # + str(submodel_uuid)
submodel["id"]      = str(submodel_uuid)

# blob: 
blob_sme = get_submodel_element_blob_template()
blob_sme["idShort"] = submodel_element_name
blob_sme["id"]      = str(submodel_element_uuid)


payload_byte = submodel_element_value.encode('utf-8')
blob_sme["contentType"] = 'application/str'
blob_sme["value"]       = base64.b64encode(payload_byte).decode('utf-8')


# append submodel element:
submodel["submodelElements"].append(blob_sme)


# post submodel into the repository
res_post_submodel = requests.post(endpoint_repository + '/submodels', auth=HTTPBasicAuth(credentials_repository["username"], credentials_repository["password"]), json=submodel)

print("{stype:12} {name:12} with UUID: {id:12}".format(stype='Submodel', name=submodel_name, id=str(submodel_uuid)))
print("{stype:12} {name:12} with UUID: {id:12}".format(stype='Submodel', name=submodel_element_name, id=str(submodel_element_uuid)))
res_post_submodel

In [ ]:
# get all submodels:
res_get_submodels = requests.get(endpoint_repository + '/submodels', auth=HTTPBasicAuth(credentials_repository["username"], credentials_repository["password"]))
print_submodels(res_get_submodels)

In [ ]:
# get subelement:
res_get_submodel = requests.get(endpoint_repository + '/submodels/' + to64str(str(submodel_uuid)) + '/submodel-elements/$value?extent=WithBlobValue', auth=HTTPBasicAuth(credentials_repository["username"], credentials_repository["password"]))
print(res_get_submodel)

print("Submodel Elements", res_get_submodel.json()['result'])
blob_sme_val = base64.b64decode(res_get_submodel.json()['result'][0][list(res_get_submodel.json()['result'][0].keys())[0]]['value']).decode('utf-8')
print("Submodel Element Value: " + blob_sme_val)

### Create corresponding Submodel Descriptor

In [ ]:
# lets add submodel descriptors for the shell descriptor (AAS):
catalog_id = submodel_uuid # to keep consistency and avoid confusion 
subprotocolBody = "id=" + str(catalog_id) + ";dspEndpoint=" + url_edc_provider_control_plane_base + "/api/v1/dsp"


register_submodel_descriptor_body = {
  "id": str(submodel_uuid),
  "semanticId": {
    "type": "ExternalReference",
    "keys": [
      {
        "type": "GlobalReference",
        "value": "test_semantic"
      }
    ]
  },
  "endpoints": [
    {
      "interface": "SUBMODEL-3.0",
      "protocolInformation": {
        "href": "url_edc_provider_data_plane" + "/submodel", # <- important, but where is the path mapping registerd?!
        "endpointProtocol": "HTTP",
        "endpointProtocolVersion": [
          "1.1"
        ],
        "subprotocol": "DSP",
        "subprotocolBody": subprotocolBody, # <- important
        "subprotocolBodyEncoding": "plain",
        "securityAttributes": [
          {
            "type":  "NONE",
            "key":   "NONE",
            "value": "NONE",
          }
        ]
      }
    }
  ]
}

# convert the id to the b64 string s.t. we can use it in the request url
b64_aas_id = to64str("urn:uuid:" + str(urn_uuid))

# add the submodel descriptor:
res_add_submodel_descriptor = requests.post(endpoint_registry + '/shell-descriptors/' +  b64_aas_id + '/submodel-descriptors', 
                                            headers=header_registry, 
                                            json=register_submodel_descriptor_body)
res_add_submodel_descriptor

In [ ]:
# view created asset
res_test_shell = requests.get(endpoint_registry + '/shell-descriptors/' +  b64_aas_id, headers=header_registry)
res_test_shell

In [ ]:
res_test_shell.json()['submodelDescriptors']

### Key Elements for the Submodel EDC registration

In [ ]:
# For the registration:
print("{edc_asset_id:15} {catalog_id}".format(edc_asset_id="edc_asset_id:", catalog_id=str(catalog_id)))
print("{submodelb64id:15} {catalog_id}".format(submodelb64id="B64-SubmodelID:", catalog_id=to64str(str(submodel_uuid))))


### Consumer View

In [ ]:
# do we even need this prviliedge consumer
registryHeader_priviledged = {
    'Content-Type': 'application/json',
    'edc-bpn': BPN_consumer,  
}